In [1]:
# from ucimlrepo import fetch_ucirepo

# # fetch dataset
# early_stage_diabetes_risk_prediction = fetch_ucirepo(id=529)

# # data (as pandas dataframes)
# X = early_stage_diabetes_risk_prediction.data.features
# y = early_stage_diabetes_risk_prediction.data.targets

In [2]:
import pandas as pd

In [3]:
from src.constants import RAW_DATA_FILE_DIR

In [4]:
df = pd.read_csv("../data/raw/raw.csv")

In [5]:
df = df.drop(columns=df.columns[0])

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder

In [7]:
df.head()

,age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [8]:
ordinal_categories = (
    df.select_dtypes(exclude="int64").drop(columns=["class", "gender"]).columns.tolist()
)

In [9]:
cat = df.select_dtypes(exclude="int64")
num = df.select_dtypes(include="int64")

In [10]:
ord_cat = [["Yes", "No"]] * len(ordinal_categories)

In [11]:
cat_ordinal = Pipeline(steps=[("Ordinal Pipeline", OrdinalEncoder(categories=ord_cat))])

In [12]:
cat_ohe = Pipeline(steps=[("One Hot Encoder", OneHotEncoder(drop="first", sparse_output=False))])

In [13]:
label_enc = Pipeline(steps=[("Label Encoder", LabelEncoder())])

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
trf = ColumnTransformer(
    transformers=[
        ("Categorical Ordinal Transformation", cat_ordinal, ordinal_categories),
        ("One Hot Transformation", cat_ohe, ["gender"]),
    ],
    remainder="passthrough",
)

In [16]:
trf

ColumnTransformer(remainder='passthrough',
                  transformers=[('Categorical Ordinal Transformation',
                                 Pipeline(steps=[('Ordinal Pipeline',
                                                  OrdinalEncoder(categories=[['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No']]))]),
                                 ['polyuria', 'polydipsia',
                                  'sudden_weight_loss', 'weakness',
                                  'polyphagia', 'genital_thrush',
                                  'visual_blurring', 'itching', 'irritability',
                                  'delayed_healing', 'partial_paresis',
                                  'muscle_stiffness', 'alopecia', 'obesity']),
                                ('One Hot Transformation',
                                 Pipeline(steps=[('One Hot Encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False))]),
                                 ['gender'])])

In [17]:
data = trf.fit_transform(df)
labels = trf.get_feature_names_out()

trf_df = pd.DataFrame(data, columns=labels)

In [18]:
le = LabelEncoder()
trf_df["remainder__class"] = le.fit_transform(trf_df["remainder__class"])

In [19]:
X = trf_df.drop(columns=trf_df.columns[-1])
y = trf_df["remainder__class"]

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [21]:
scaler = StandardScaler()

In [30]:
X_train = scaler.fit_transform(X_train)

In [35]:
import numpy as np

pd.DataFrame(np.hstack([X_train, y_train.values.reshape(-1, 1)]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.982905,0.870388,0.800943,1.169336,-1.090266,0.506278,-1.053134,1.017393,0.550791,0.917208,0.845154,-1.248529,0.758064,0.419137,0.758064,0.728742,0.0
1,-1.017393,-1.148913,-1.248529,-0.855186,-1.090266,-1.975198,-1.053134,-0.982905,0.550791,-1.090266,0.845154,-1.248529,-1.319150,0.419137,0.758064,0.485440,1.0
2,-1.017393,-1.148913,0.800943,-0.855186,-1.090266,0.506278,0.949547,-0.982905,0.550791,0.917208,-1.183216,-1.248529,0.758064,0.419137,-1.319150,1.377548,1.0
3,0.982905,0.870388,-1.248529,-0.855186,0.917208,0.506278,-1.053134,-0.982905,0.550791,-1.090266,0.845154,0.800943,0.758064,0.419137,-1.319150,0.404339,1.0
4,0.982905,0.870388,-1.248529,-0.855186,-1.090266,0.506278,-1.053134,-0.982905,0.550791,-1.090266,0.845154,-1.248529,-1.319150,0.419137,0.758064,0.647641,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,0.982905,-1.148913,-1.248529,-0.855186,0.917208,0.506278,0.949547,-0.982905,0.550791,-1.090266,-1.183216,-1.248529,0.758064,0.419137,-1.319150,-1.055476,1.0
344,0.982905,-1.148913,-1.248529,-0.855186,-1.090266,0.506278,-1.053134,-0.982905,0.550791,0.917208,-1.183216,0.800943,-1.319150,-2.385856,0.758064,0.809843,1.0
345,-1.017393,-1.148913,-1.248529,-0.855186,0.917208,0.506278,-1.053134,1.017393,0.550791,-1.090266,-1.183216,-1.248529,0.758064,0.419137,-1.319150,-0.649972,1.0
346,-1.017393,-1.148913,-1.248529,-0.855186,-1.090266,0.506278,-1.053134,1.017393,0.550791,0.917208,-1.183216,0.800943,0.758064,0.419137,0.758064,0.728742,1.0


In [36]:
pd.DataFrame(np.hstack([X_test, y_test.values.reshape(-1, 1)]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,72,0
1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,40,1
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,57,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,41,1
4,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,45,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,51,0
168,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,54,1
169,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,48,1
170,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,45,0


In [ ]:
final_pipeline = Pipeline([("scaler", StandardScaler()), ("classifier", LogisticRegression())])

In [ ]:
model = final_pipeline.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_test)

In [ ]:
from box import ConfigBox

In [ ]:
transformations = ConfigBox(model.named_steps)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(y_test, pred)

0.9418604651162791

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92        61
           1       0.95      0.95      0.95       111

    accuracy                           0.94       172
   macro avg       0.94      0.94      0.94       172
weighted avg       0.94      0.94      0.94       172



In [ ]:
pd.concat([X_train, y_train], axis=1)

,Categorical Ordinal Transformation__polyuria,Categorical Ordinal Transformation__polydipsia,Categorical Ordinal Transformation__sudden_weight_loss,Categorical Ordinal Transformation__weakness,Categorical Ordinal Transformation__polyphagia,Categorical Ordinal Transformation__genital_thrush,Categorical Ordinal Transformation__visual_blurring,Categorical Ordinal Transformation__itching,Categorical Ordinal Transformation__irritability,Categorical Ordinal Transformation__delayed_healing,Categorical Ordinal Transformation__partial_paresis,Categorical Ordinal Transformation__muscle_stiffness,Categorical Ordinal Transformation__alopecia,Categorical Ordinal Transformation__obesity,One Hot Transformation__gender_Male,remainder__age,remainder__class
274,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,57,0
426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,54,1
60,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,65,1
110,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,53,1
329,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,35,1
106,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,58,1
270,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,40,1
435,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,57,1
